# 05 · Importación de datos en diferentes formatos

**Módulo I — Fundamentos**

## Objetivos:
- Importar datos desde diferentes formatos: CSV, Excel, JSON, XML
- Manejar problemas comunes de importación (encodings, separadores, tipos)
- Exportar datos procesados a múltiples formatos
- Aplicar técnicas de validación y limpieza inicial
- Trabajar con APIs y datos web en tiempo real

## Requisitos:
- pandas, openpyxl, lxml, requests instalados
- Conexión a internet para ejemplos con APIs
- Datos de ejemplo (se generarán automáticamente)

## Índice:
1. Preparación del entorno y datos de ejemplo
2. Importación desde CSV
3. Trabajando con archivos Excel
4. Manejo de datos JSON
5. Procesamiento de archivos XML
6. Conexión con APIs médicas
7. Exportación a múltiples formatos
8. Ejercicios prácticos con datos médicos

## 1️⃣ Preparación del entorno y datos de ejemplo

Primero verificamos las librerías necesarias y creamos datos sintéticos para practicar.

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import json
import requests
import xml.etree.ElementTree as ET
from pathlib import Path
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configurar pandas para mejor visualización
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 1000)

print("Librerías importadas exitosamente")
print(f"Pandas versión: {pd.__version__}")
print(f"NumPy versión: {np.__version__}")

# Crear estructura de carpetas para datos
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / "data"
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"
EXAMPLES_DIR = DATA_DIR / "examples"

for dir_path in [RAW_DIR, PROCESSED_DIR, EXAMPLES_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)

print(f"\nEstructura de carpetas creada:")
for dir_path in [RAW_DIR, PROCESSED_DIR, EXAMPLES_DIR]:
    print(f"  ✅ {dir_path.relative_to(BASE_DIR)}")

In [ ]:
# Generar datos sintéticos para ejemplos médicos
def generar_datos_medicos_sinteticos(n_pacientes=100):
    """
    Genera un dataset sintético de pacientes para practicar importación/exportación.
    """
    np.random.seed(42)
    
    # Generar datos base
    nombres = ["Ana", "Carlos", "María", "Luis", "Sofia", "Pedro", "Laura", "Miguel", "Carmen", "Diego"]
    apellidos = ["García", "Rodríguez", "López", "Martínez", "González", "Pérez", "Sánchez", "Ramírez"]
    ciudades = ["Bogotá", "Medellín", "Cali", "Barranquilla", "Cartagena"]
    tipos_sangre = ["O+", "A+", "B+", "AB+", "O-", "A-", "B-", "AB-"]
    especialidades = ["Medicina General", "Cardiología", "Neurología", "Pediatría", "Ginecología"]
    
    # Generar fechas de consulta en los últimos 2 años
    fecha_inicio = datetime.now() - timedelta(days=730)
    fechas_consulta = [fecha_inicio + timedelta(days=np.random.randint(0, 730)) for _ in range(n_pacientes)]
    
    # Crear dataset
    data = {
        "id_paciente": range(1, n_pacientes + 1),
        "nombre": np.random.choice(nombres, n_pacientes),
        "apellido": np.random.choice(apellidos, n_pacientes),
        "edad": np.random.randint(18, 85, n_pacientes),
        "genero": np.random.choice(["M", "F"], n_pacientes),
        "ciudad": np.random.choice(ciudades, n_pacientes),
        "tipo_sangre": np.random.choice(tipos_sangre, n_pacientes),
        "peso": np.round(np.random.normal(70, 15, n_pacientes), 1),
        "altura": np.round(np.random.normal(170, 10, n_pacientes), 0),
        "presion_sistolica": np.random.randint(90, 180, n_pacientes),
        "presion_diastolica": np.random.randint(60, 110, n_pacientes),
        "especialidad_consulta": np.random.choice(especialidades, n_pacientes),
        "fecha_consulta": fechas_consulta,
        "costo_consulta": np.random.randint(80000, 350000, n_pacientes)
    }
    
    df = pd.DataFrame(data)
    
    # Calcular IMC
    df['imc'] = np.round(df['peso'] / (df['altura'] / 100) ** 2, 2)
    
    # Formatear fecha como string para mejor compatibilidad
    df['fecha_consulta'] = df['fecha_consulta'].dt.strftime('%Y-%m-%d')
    
    print(f"Dataset sintético generado: {df.shape[0]} pacientes, {df.shape[1]} columnas")
    print(f"\nColumnas: {list(df.columns)}")
    
    return df

# Generar dataset principal
df_pacientes = generar_datos_medicos_sinteticos(100)
print(f"\nPrimeras 5 filas:")
display(df_pacientes.head())

## 2️⃣ Importación desde CSV

CSV (Comma Separated Values) es el formato más común para intercambio de datos tabulares.

In [ ]:
# Crear diferentes variaciones de CSV para practicar
def crear_csvs_ejemplo():
    """
    Crea varios archivos CSV con diferentes características para practicar.
    """
    # CSV básico (separado por comas)
    csv_basico = EXAMPLES_DIR / "pacientes_basico.csv"
    df_pacientes.to_csv(csv_basico, index=False)
    
    # CSV con punto y coma (común en Excel español)
    csv_semicolon = EXAMPLES_DIR / "pacientes_semicolon.csv"
    df_pacientes.to_csv(csv_semicolon, index=False, sep=';')
    
    # CSV con encoding Latin1 (problemas comunes)
    csv_latin1 = EXAMPLES_DIR / "pacientes_latin1.csv"
    df_pacientes.to_csv(csv_latin1, index=False, encoding='latin1')
    
    # CSV con datos faltantes simulados
    df_con_nan = df_pacientes.copy()
    # Simular datos faltantes en algunas columnas
    mask_peso = np.random.random(len(df_con_nan)) < 0.1
    mask_presion = np.random.random(len(df_con_nan)) < 0.05
    df_con_nan.loc[mask_peso, 'peso'] = np.nan
    df_con_nan.loc[mask_presion, 'presion_sistolica'] = np.nan
    
    csv_nan = EXAMPLES_DIR / "pacientes_con_nan.csv"
    df_con_nan.to_csv(csv_nan, index=False)
    
    archivos_creados = [
        (csv_basico, "CSV básico (comas, UTF-8)"),
        (csv_semicolon, "CSV con punto y coma"),
        (csv_latin1, "CSV con encoding Latin1"),
        (csv_nan, "CSV con datos faltantes")
    ]
    
    print("Archivos CSV creados para práctica:")
    for archivo, descripcion in archivos_creados:
        tamaño = archivo.stat().st_size / 1024  # KB
        print(f"  ✅ {archivo.name:<25} - {descripcion} ({tamaño:.1f} KB)")
    
    return archivos_creados

archivos_csv = crear_csvs_ejemplo()

## ✅ Resumen del Notebook

En este notebook hemos dominado la importación de datos en Python:

### Formatos cubiertos:
1. ✅ **CSV**: Separadores, encodings, tipos de datos, valores faltantes
2. ✅ **Excel**: Múltiples hojas, rangos específicos, formateo
3. ✅ **JSON**: Estructuras simples y anidadas, normalización
4. ✅ **XML**: Parseo, extracción de datos médicos estructurados
5. ✅ **APIs**: Conexión, procesamiento, almacenamiento

### Técnicas aprendidas:
- Manejo de diferentes encodings (UTF-8, Latin1)
- Optimización de tipos de datos para memoria
- Normalización de JSON anidados
- Parseo de XML con ElementTree
- Conexión a APIs con requests
- Exportación con configuraciones específicas

### Aplicaciones médicas:
- **Registros de pacientes** desde sistemas hospitalarios
- **Resultados de laboratorio** en formato XML estándar
- **APIs de signos vitales** en tiempo real
- **Reportes epidemiológicos** en Excel multi-hoja
- **Intercambio de datos** entre sistemas de salud

### Próximos pasos:
- **Módulo II:** Conexión y gestión de bases de datos
- **Módulo III:** Análisis exploratorio y limpieza de datos
- **Módulo IV:** Visualización y reportes automatizados

---

**¡Dominar la importación de datos es fundamental para cualquier proyecto de análisis! Con estas técnicas podrás trabajar con datos de cualquier sistema médico o hospitalario.**